__Lab 01 - Acquisition__

Student Names: Ganesh Veluru and Jwala Sri Hari Badam

Group Number: 3

# Setup

## FPGA Cofigurations & Import Python Libraries

These are the FPGA configurations need when programming it with the bitstream.

BITSTREAM_FILENAME is the path and filename to the Bitstream to program the FPGA with.

Also, we import the Python libraries here.

In [31]:
import numpy as np
import serial
import serial.tools.list_ports
from tqdm import tnrange

DEVICE_NAME = "CmodS7"
DEVICE_ID   = 0
BITSTREAM_FILENAME = "../Hardware/APUF_CmodS7_01.bit" # For both undergrad & grad students

# BITSTREAM_FILENAME = "../Hardware/APUF_CmodS7_02.bit" # For grad students only

print("✔️ OK to continue!")

✔️ OK to continue!


## Connect to FPGA

We need to make sure the FPGA is connected.

"%%bash" indicated that the following code in that cell will run with "bash" instead of python.

In [32]:
!djtgcfg enum

print("✔️ OK to continue!")

Found 1 device(s)

Device: CmodS7
    Device Transport Type: 00020001 (USB)
    Product Name:          Digilent Cmod S7
    User Name:             CmodS7
    Serial Number:         210376B5B57A
✔️ OK to continue!


Make sure to that it says "Found 1 device(s)" (or more) and the device name match with "DEVICE_NAME" from section 1.1

Also, we can verify the device ID (DEVICE_ID) from section 1.1 with the following code.

(DEVICE_ID should be 0 for CmodS7, unless you have other FPGA connected)

In [33]:
%%bash -s "$DEVICE_NAME"
djtgcfg init -d $1

Initializing scan chain...
Found Device ID: 037c4093

Found 1 device(s):
    Device 0: XC7S25


The actual device ID is located in the list at the bottom. It should say, for example:

<span class="mark">Found 1 device(s):</span> <br>
<pre> <span class="mark">Device 0: XC7S25</span>

The 0 here is the Device ID for "XC7S25" (i.e., CmodS7).

## Program the FPGA

Program the FPGA with the bitstream.

Also, make sure the BITSTREAM_FILENAME in section 1.1 matches with bitstream you want to programm the FPGA with.

In [34]:
%%bash -s "$DEVICE_NAME" "$DEVICE_ID" "$BITSTREAM_FILENAME"
djtgcfg prog -d $1 -i $2 -f $3

Programming device. Do not touch your board. This may take a few minutes...
Programming succeeded.


## Setup the USB serial communication port

Now, we can setup PySerial.

First, let's identify the USB port name where the FPGA is connected to. Use the function below to print a list of all your ports.

In [35]:
def print_ports():
    portsList = serial.tools.list_ports.comports()

    print("--- PORTS LIST ---")
    for p in sorted(portsList):
        # first prints the port, then the device product
        print(f'{p[0]} : {p[1]}')

print_ports()

print("✔️ OK to continue!")

--- PORTS LIST ---
/dev/ttyUSB1 : Digilent Adept USB Device - Digilent Adept USB Device
✔️ OK to continue!


In [41]:
# connect to COM port, set baudrate
port = "/dev/ttyUSB1"
boudrate = 115200
ser = serial.Serial(port, boudrate)

print("✔️ OK to continue!")

✔️ OK to continue!


load data from npz file, loads as directory with one item 'challenge'

In [42]:
in_filename = "../Challenges/challenges_05k.npz" # for testing only
#in_filename = "../Challenges/challenges_10k.npz"
# in_filename = "../Challenges/challenges_50k.npz" # final results must use this

data = np.load(in_filename)
challenges = data['challenge']
print((challenges))

print("✔️ OK to continue!")

['e66fdeb312e9754f' '0a020391ad0019e4' 'ac5394f69da92950' ...
 '26c818120d0ac00b' 'c7adc10603770d1f' 'b56c01b60829760c']
✔️ OK to continue!


# Send & Collect Data

Let's make a functions that will send 1 challenge and receive 1 response

__Complete missing parts__

In [43]:
# Function: get_1resp()
#   Input: 'ser' - serial connection to FPGA
#   Input: 'in_hex' - an input hex string to send to the FPGA
#   Input: 'resp_size' - size of the output response in bytes
#   Output: 'r_bin' - respective binary string response of size 'resp_size' bytes (8*resp_size bits).


def get_1resp(ser: serial.Serial, in_hex: str, resp_size: int) -> str:

    # convert hex string to bytearray
    in_bytes = bytearray.fromhex(in_hex)

    # send bytes to FPGA
    ser.write(in_bytes)
    
    # get response: reads n bytes from FPGA (remember: 1 byte = 8 bits)
    r_bytes = ser.read(resp_size)
    # convert bytearray to int
    r_int = int.from_bytes(r_bytes, byteorder='big')
    # convert int to resp_size*8 bits binary string
    r_bin = format(r_int, "0"+str(8*resp_size)+"b")
    
    return r_bin

print("✔️ OK to continue!")

✔️ OK to continue!


Test this function. Send an example challenge and return that value. The response size is 2 bytes.

In [44]:
# Example usage:
# Replace "/dev/ttyUSB1" with the correct COM port name, and 115200 with the desired baud rate

response = get_1resp(ser, str(data['challenge'][0]), 2)  # Example: sending "1A2B3C" hex and expecting a 2-byte response
# print("Response:", response)
# ser.close()  # Close the serial connection when done

bytearray(b'\xe6o\xde\xb3\x12\xe9uO')
b'\xff\xba'
65466


Now, lets place this function in a loop to iterate all the challenges and get all the responses.

note: tnrange() is a fancy way to do loops. This is similar to range() function but makes a progress bar. It is nice to use when loading a lot of data.

__Complete missing parts__

In [45]:
# Function: get_all_resp()
#   Input: 'challanges' - NumPy array of challanges to send to the FPGA
#   Input: 'resp_size' - size of each output response in bytes
#   Output: 'responses' - NumPy array respective binary string response of size 'resp_size' bytes.

def get_all_resp(challenges: np.ndarray, resp_size: int):
    # initialize an empty list (or NumpPy array)
    L=[]
    
    # iterate through each challenge and get respective response
    N = len(challenges)
    for i in tnrange(N , desc='Generating APUF Responses'):
        L.append(get_1resp(ser,challenges[i],2))
        # do something
    
    #print(type(L))
    # If you used a list, convert responses to Numpy Array
    responses=np.array(L)
    # return your NumPy array of responses
    return responses

Now we can generate the responses

In [46]:
responses = get_all_resp(challenges, 2)

print("✔️ OK to continue!")


/tmp/ipykernel_5573/3816777311.py:12: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(N , desc='Generating APUF Responses'):


Generating APUF Responses:   0%|          | 0/5000 [00:00<?, ?it/s]

bytearray(b'\xe6o\xde\xb3\x12\xe9uO')
b'\x00\xe5'
229
bytearray(b'\n\x02\x03\x91\xad\x00\x19\xe4')
b'\xa6\xf9'
42745
bytearray(b'\xacS\x94\xf6\x9d\xa9)P')
b'\x87\xaa'
34730
bytearray(b'\x18\x1f\x1a6\xac\xc2\xf6\xd6')
b'\xba{'
47739
bytearray(b'\xa5\xcb\xf9\x9f*t \x00')
b'Z\xa5'
23205
bytearray(b'\x0bm\xae\x00\xf7<\xb4,')
b'\x8f1'
36657
bytearray(b'7i\xf8\xfa\x81w\xd5\xa4')
b'O\xf8'
20472
bytearray(b'$\xcf\xa8\xc1\xcb}\x13\x8b')
b'\x91\xc4'
37316
bytearray(b'\xd2\x94!\xee\xf4\xe8\x9c\xb2')
b'\x9b\xf1'
39921
bytearray(b'\x8b\x00\x95\xee\xdd\x1c\xd0\xb0')
b'dz'
25722
bytearray(b'\x98\x96\xc4\x94*G\xa8\xca')
b'\x84\x1e'
33822
bytearray(b'm\xd2\x0cR\xe7\xddP\xfd')
b'V\xbf'
22207
bytearray(b's\x05<\xa4\x01\xca\xfc\xff')
b'h\x01'
26625
bytearray(b'C\r\xc4\xe0.w\xf4\x9a')
b'?\xd8'
16344
bytearray(b'\xd2\x9c\x96\x83h\xd6u\x8a')
b'\x92\xc7'
37575
bytearray(b'\xaf\xf9\xf3\xfa\xa7Q\x08.')
b'\x00\x05'
5
bytearray(b'(\xa6\xd20\xa7\\;#')
b'\xbe\xf6'
48886
bytearray(b'\x89|\x16\xfe\x9dl\x9c\x95')
b'U\

b'\xb9\xee'
47598
bytearray(b'\x85)\x1f\xafwk\x92P')
b'\xd2\xc8'
53960
bytearray(b'O\xf0\x8d\xf4FWl\x85')
b']\x80'
23936
bytearray(b'\x12\x01\x0f?\xb7\xdas\x9c')
b'\x04%'
1061
bytearray(b'y\xb9\xe8\x0e\xa1R\x18\xe2')
b'@\xe1'
16609
bytearray(b'\xe9\xc8\x1c{6\x03\xa3\xa5')
b'\xfaI'
64073
bytearray(b'^,.\xff\x88\x054"')
b'\xff\x80'
65408
bytearray(b'rI\xa1i\xf6\x96\xf5\xed')
b'\xf9\xfc'
63996
bytearray(b'/\xc5\xd5\xe9r\x92\x90P')
b'Ea'
17761
bytearray(b'M9\xdb\xacy\xc4r\xbf')
b'\xe6\xb7'
59063
bytearray(b'\x91JB\xb0\x06\t\xb3\x18')
b'\xfe:'
65082
bytearray(b'\xf5\x06\x1b\xcf\xb4B\xe8M')
b'\xa2\x82'
41602
bytearray(b'\xd3goH\xf0t\xb1N')
b'\x7f\xad'
32685
bytearray(b'\xe6\x1b\xcc\x0b\xc9yi"')
b'\xbf9'
48953
bytearray(b'k\x1e5\x92\xd8\xfb\x1cH')
b'\xed\xac'
60844
bytearray(b'\x02\xe1\xa9\xa7\xca\xd9rY')
b'\xa6\xad'
42669
bytearray(b'\xb6\xeb\x92-vE\xc3\xcd')
b'\xc39'
49977
bytearray(b'\xbe\x7fG\xad\x8b\x91g\x8d')
b'\n('
2600
bytearray(b'\xbb\x0bk\\\x9d\xfd\xec\xa9')
b'\xf8\xcd'
63693
bytear

b'\xd9<'
55612
bytearray(b'~}z#K\xec\x1bF')
b'\x7f\xf8'
32760
bytearray(b'\xeb*wVz\xdc\xbc\x10')
b'\xff~'
65406
bytearray(b'\x82\xdc\x10\xaaG\x85\xbc>')
b'\x03C'
835
bytearray(b'\xa55_\x7fT\xba\xc6\xe4')
b'\xa1\xdb'
41435
bytearray(b'\xfd7,\x82\x99\x9bN\xce')
b'J['
19035
bytearray(b'\xc3\x0cQ\x1a\x01\x9c\xdf\x81')
b'\xcd9'
52537
bytearray(b'34r\xc0j\x9f\xf9]')
b'\xcf\xff'
53247
bytearray(b'`\x07\x91!H\xb5\x7fN')
b'_\x80'
24448
bytearray(b'\xffB\xda\xd62\x83A\x12')
b'\x01I'
329
bytearray(b'\x00\xdc\x90k3g\xb1\xa5')
b'\xd9\n'
55562
bytearray(b'\xdch\xf0U\x19\xbf\xbb\xad')
b'\x07\x00'
1792
bytearray(b'+{\x04mC\xael\xdd')
b'#Q'
9041
bytearray(b'\x8bb\x05\xf1\x97E+\xcc')
b'\xaf\xb4'
44980
bytearray(b'\xd8Dh\xf9\xb7\x13\xabr')
b'\xbe\xa2'
48802
bytearray(b'(\xbf\xa7\x0b\x8bXc\x00')
b'\xbf\x00'
48896
bytearray(b'\xcf\x98^W\xe9\xd8\xd4>')
b'\xfe\xbd'
65213
bytearray(b'\x9f\xce\xe9J\xc6\x08\xcf\xdd')
b'\xff\xff'
65535
bytearray(b'\x0c*\x11\x08\xa2L"\x0b')
b'\xf7\xf8'
63480
bytearray(b'\x93\x1aQ

...

b'\x87\xb8'
34744
bytearray(b'\x10k\xc0\x9f\xac\x1e\xa0\xef')
b'\xf3^'
62302
bytearray(b'\xc7!\xd7\xca\x91m\xf5\xb9')
b'\x00\x07'
7
bytearray(b'7\x07\x81\x89\xb1\xd9\x02\xe4')
b'\xdf\xf4'
57332
bytearray(b'Y3zb\x17<+C')
b'\xa8,'
43052
bytearray(b'y\xc7\xa0\xa9\xb7\xdb\xd9\x8d')
b'\xfar'
64114
bytearray(b'\xf7\x8a\xcc\xbd\xc7\x84\x03\xc9')
b'\xf1\xd6'
61910
bytearray(b"\xdcw\xb0\x05\x0fs\'\x0e")
b'\xd3\r'
54029
bytearray(b'_\xb7\xe5\xa2D\xbd\x12\x93')
b'\xff\xf4'
65524
bytearray(b'\xcbP\xd9*\x0f\xf0\x7fm')
b'})'
32041
bytearray(b':\\\xb9\x92\xc9\xe2\xe4{')
b'\x83\t'
33545
bytearray(b'\xf7uz\xd6\x9cy\xe3\xf0')
b'\x9f\x12'
40722
bytearray(b'\x93\x16\x12\xec\xc3\x01Q7')
b'\xc5Y'
50521
bytearray(b'\xbb\x9e\x17\xe9\xc2u\xd6,')
b'\x19\x01'
6401
bytearray(b'\xae\xc4\x7f\x977QR\x9b')
b'\xb7\xa0'
47008
bytearray(b'\xc2\x93P\xa2&}\x11q')
b'\xbb\xf8'
48120
bytearray(b'\xc8\x97\xa0rtPS%')
b'\xbf\xb6'
49078
bytearray(b'\x05\xdbj\xe8\x11q\n\x9a')
b'W\xb2'
22450
bytearray(b'\xa6\xeb\xc8\x8c\xcb\xd6\xa

b'\x10\x01'
4097
bytearray(b'+\x83\x14>\xe89\xff\x92')
b'\x99\r'
39181
bytearray(b'\xd4\xba\x0c\xe5\xd9bo\xc2')
b'd|'
25724
bytearray(b'\xec\xbaxtH\xeb\xfc\x7f')
b'\xee\xbc'
61116
bytearray(b'\xd3\xb6|\xff\xcb\xc1W\\')
b'\x0c>'
3134
bytearray(b'\x17\x1diS3\xa2\x9a#')
b'$a'
9313
bytearray(b'\xed&\xe8\xf7\x13p\x96\xdc')
b'\xe6\xb3'
59059
bytearray(b'#i\x0f\x86\xc7e\xfb\xc1')
b'DA'
17473
bytearray(b'\x166iL-\xef%\xcf')
b'^\xf7'
24311
bytearray(b'\xcc4\xfdYN\x9b\xe0 ')
b'\xb3\xf6'
46070
bytearray(b'\xb5\x8b\x81\x03\x17~\x18\xc7')
b'\xd7\xd6'
55254
bytearray(b'F\xa4\xa3\x1aY\xb0\x84\xd3')
b'\xeek'
61035
bytearray(b'\x02B\x10\xe7\x98\xb3\x01s')
b'\xff\x00'
65280
bytearray(b'\x13\xaa\x17Z\x07J@\xd4')
b'\xee\xff'
61183
bytearray(b'+\xb6\xee\x90\x88X\x16\xc6')
b'\x95\x1e'
38174
bytearray(b'O\xf2\xd3l\xc9\x83\xbbn')
b'\xbf\xee'
49134
bytearray(b'z\xf1>\xc3\x03u\x7f\x9b')
b'"\x81'
8833
bytearray(b'\x00\x9b\x94\x0f\xfb\x02\x1f\x89')
b'!\xf9'
8697
bytearray(b'M\x8f"\xb7\x02/c(')
b'\xdb\xfd'
56317
b

b'Q\x9a'
20890
bytearray(b"\xbe\'\x96\xc1u\x86\xef\xbc")
b'\xff\xf6'
65526
bytearray(b"A\x97\xf6\xce)\'\x11,")
b'\xfb\xef'
64495
bytearray(b'\x14\x82#\xe6\x06\xfb"t')
b'\x85\x8d'
34189
bytearray(b'\xcc\xd1\x82:\xf6\xdd\xbf\xbe')
b'\xcd\x9a'
52634
bytearray(b'Y\x99\xfd\xf95-f\x97')
b'\x00I'
73
bytearray(b'\xf8M\xb5\x9f\xbc ?\x10')
b'\xef\xff'
61439
bytearray(b'\x00`5\xdc\xfb\xebG\x96')
b'\x7f~'
32638
bytearray(b'\xd9<[\x96\xd0GN\xa7')
b'\x8e:'
36410
bytearray(b'\xccrM\xda\xa6Ie%')
b'\xc89'
51257
bytearray(b'\x1b\x02\x8e\xdd\xc33\x01(')
b'3\x81'
13185
bytearray(b'\xd6WN\x02\xd5\xfa\x9d\xf9')
b'\x1a\x81'
6785
bytearray(b'~\x07\xa0i\xdf\x0b\xa2X')
b'~\xfc'
32508
bytearray(b'\x14(zlp\xeb1@')
b'6\x8f'
13967
bytearray(b'D`\n\x92\x94\x84\x1a\x1b')
b'\xa8\xa4'
43172
bytearray(b'%\xd8?\xcd\x10xK\x07')
b'\xbf\xe6'
49126
bytearray(b'\x1b\xe2o\xd3*\x81\xac#')
b'|\xa5'
31909
bytearray(b'\xd7ea\xe7\xf4\\\xa4\xa3')
b'\x16\x97'
5783
bytearray(b'\xb6\xa8\xe3\xa5M\xb7\xb2\x01')
b'\x02\xfb'
763
bytearray(

b'\xfd\xf0'
65008
bytearray(b'\x8f\x89\xdb\xbaXPh\xf2')
b'\xca;'
51771
bytearray(b"\xf0\xb4\nY>\x8b\xad\'")
b'\xb1\xf2'
45554
bytearray(b'\xa8\x14\x85\xd62\xf2\xd8\x93')
b'\x05\xbc'
1468
bytearray(b'\xc9\x08\xd9\x01%\x7f\xcd\x9e')
b'\x8d\x85'
36229
bytearray(b'\x7f\xe9\xe6\x048\x12\xdb\r')
b'M\xdf'
19935
bytearray(b'P\x12(\x0c\xb7\x1eu\xdc')
b'\xe2\x05'
57861
bytearray(b'\x06n-\xe6_\xec\x14s')
b'd\xf9'
25849
bytearray(b'L\xfe\x02\x14\xe7Y\x01,')
b'\x02I'
585
bytearray(b'\x90\x1a\xf1\x0b\xf7=\xb3_')
b'\xa1\xf3'
41459
bytearray(b'$X\x85\xb9Q\xb89N')
b'\x01\xe0'
480
bytearray(b'\xe2\x9d)\xa4\x98t3d')
b'\xee\xfe'
61182
bytearray(b'\xc4\x99\x05\xe6Q\x01\xdev')
b'\x14\xa4'
5284
bytearray(b'5\xdet\xc5\xcf\x0c\xe1k')
b'\xd3\x00'
54016
bytearray(b'\x02\x0cJ\xfc\xe9\x92\xd8\x8b')
b'\xf7\xda'
63450
bytearray(b'\xd3\xce\xaaD\xd6\xffa\xb4')
b'<\r'
15373
bytearray(b'\x8d\x07\xc1F\xc2V\xa0\xcd')
b'\xf9\x00'
63744
bytearray(b'\x06\x15\xbe\xee=;\xe6\x9d')
b'j\x07'
27143
bytearray(b'\x98\x00\xd9\xa1\xe7

b'~\xc1'
32449
bytearray(b'\xcd\xe0m\x0c\x80\x81\xfe\x96')
b'\xa7\x95'
42901
bytearray(b'\x1e6q\xda\x8c0]\x8a')
b'?\xf4'
16372
bytearray(b'`\xe2\xbf\x9d\xa3#;\xcf')
b'\xbf\xfc'
49148
bytearray(b'n2\xd9y\xa1x\xc1z')
b'\x86\xf6'
34550
bytearray(b'@\xd0\x82\x04G\x00\x1ap')
b'Z\xf6'
23286
bytearray(b'\xe8\xec\xfc\x83\xed-q\xca')
b'\xb7\xbe'
47038
bytearray(b'\xa1l\x0e\xedy5\xb54')
b'U9'
21817
bytearray(b'E0\xee1\x11\x9eg ')
b'\x85\x18'
34072
bytearray(b'\xb8\x94\xa2\xce\x89\x18\xd0\x88')
b'\x00\xa3'
163
bytearray(b'\xbc9:/\x8b\xa8k\xc9')
b'\xb5\xd5'
46549
bytearray(b'*b4x\xee\xa7\xdb,')
b'\x8c$'
35876
bytearray(b'\x84"Tm\xa6\xb3!\xa1')
b'X)'
22569
bytearray(b'C9\xd1\x1d[\xfb/\x01')
b'\xb0J'
45130
bytearray(b'f\xb6\x9a[\x82\x11\xef\xb9')
b'\xfc\x8e'
64654
bytearray(b',+\xbd\x1d\xad:\r\xf5')
b'Y\x01'
22785
bytearray(b'c\n\xe6\xfeuq\xf3\x99')
b'\xfd\x00'
64768
bytearray(b'\xc9\xc15\xae\xd1\xee\xf8\x89')
b'\xff\xbc'
65468
bytearray(b'M\xda\xc8\xf1Q)\xd4B')
b'~\x00'
32256
bytearray(b'\xd3\x03\x

Now, lets save the output on a npz file.

In [47]:
# output list into npz file, save as directory with one item 'response'
print(type(responses))
out_filename = "../Responses/fpga1_resp0_05k_new.npz"
np.savez(out_filename, response=responses)
print('Output File =', out_filename)

print("✔️ OK to continue!")

<class 'numpy.ndarray'>
Output File = ../Responses/fpga1_resp0_05k_new.npz
✔️ OK to continue!


Re-run these functions to generate all 6 responses: 3 for each FPGA. Make sure to rename the output file adequatly to represent each varying file.